In [1]:
#%pip install bertopic

In [2]:
# %pip install pandas
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm import tqdm

2025-06-26 12:46:10.728309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-26 12:46:10.864145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-26 12:46:10.864161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-06-26 12:46:10.887749: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-26 12:46:11.582511: W tensorflow/stream_executor/platform/de

In [3]:
from platform import python_version

print(python_version())

3.8.10


In [4]:
df = pd.read_parquet("refined2.parquet")

In [12]:
df

,date,tweet,topic,field
0,2021-06-20 05:26:01,What can be done? - Never blindly trust an ab...,15,0
1,2021-06-20 04:46:01,"""We need a paradigm shift from model-centric t...",15,0
2,2021-06-20 04:30:00,Using high-resolution satellite data and compu...,15,0
3,2021-06-20 04:19:01,.@Stephenson_Data shares four steps that will ...,15,0
4,2021-06-20 02:40:01,"""Curricula is inherently brittle in a world wh...",15,0
...,...,...,...,...
33585,2010-01-06 06:13:47,Need data.gov.sg now! RT @jranck: Chart Wars: ...,15,2
33586,2010-01-06 03:24:09,Great presentation by @alexlundry: Chart Wars:...,15,2
33587,2010-01-06 00:29:44,RT @briandevine: u should also check out the d...,15,2
33588,2010-01-05 19:45:15,New Data Visualization - Application performan...,15,2


In [6]:
df2 = pd.read_csv('data-science-tweets/tweets/data_visualization.csv', usecols=['date', 'time', 'tweet'])

In [7]:
df2.loc[:,'date'] = pd.to_datetime(df2.date.astype(str)+' '+df2.time.astype(str))

In [8]:
df2 = df2.drop(columns=['time'])
df2

,date,tweet
0,2021-06-20 01:10:03,Take your storytelling to the next level using...
1,2021-06-19 22:17:46,Choosing Fonts for Your Data Visualization | b...
2,2021-06-19 08:21:55,This data visualization shows where our greate...
3,2021-06-18 23:29:21,Looking for examples of stellar charts made so...
4,2021-06-18 23:05:03,"With #WISQARS Data Visualization, you can disp..."
...,...,...
33585,2010-01-06 06:13:47,Need data.gov.sg now! RT @jranck: Chart Wars: ...
33586,2010-01-06 03:24:09,Great presentation by @alexlundry: Chart Wars:...
33587,2010-01-06 00:29:44,RT @briandevine: u should also check out the d...
33588,2010-01-05 19:45:15,New Data Visualization - Application performan...


In [9]:
df.to_parquet('refined2.parquet')

In [13]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

    
MODEL = f"cardiffnlp/tweet-topic-base-multilingual"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

text = "Israeli drone strikes"
tokens = tokenizer(text, return_tensors='pt')
output = model(**tokens)

scores = output[0][0].detach().numpy()
scores = expit(scores)
predictions = (scores >= 0.5) * 1

In [14]:
inverse_class_mapping = {val: key for key, val in class_mapping.items()}
inverse_class_mapping

{'arts_&_culture': 0,
 'business_&_entrepreneurs': 1,
 'celebrity_&_pop_culture': 2,
 'diaries_&_daily_life': 3,
 'family': 4,
 'fashion_&_style': 5,
 'film_tv_&_video': 6,
 'fitness_&_health': 7,
 'food_&_dining': 8,
 'gaming': 9,
 'learning_&_educational': 10,
 'music': 11,
 'news_&_social_concern': 12,
 'other_hobbies': 13,
 'relationships': 14,
 'science_&_technology': 15,
 'sports': 16,
 'travel_&_adventure': 17,
 'youth_&_student_life': 18}

In [15]:
fe_pipeline = pipeline(
    task="text-classification",
    model=MODEL,
    torch_dtype=torch.float16,
    device=0
)

In [7]:
output = [inverse_class_mapping[fe_pipeline(t)[0]['label']] for t in df2['tweet']]
print(len(output))

NameError: name 'df2' is not defined

In [25]:
df2['topic'] = output
#df1.to_parquet('refined2.parquet')

In [26]:
df2['field'] = 2

In [27]:
df2

,date,tweet,topic,field
0,2021-06-20 01:10:03,Take your storytelling to the next level using...,10,2
1,2021-06-19 22:17:46,Choosing Fonts for Your Data Visualization | b...,15,2
2,2021-06-19 08:21:55,This data visualization shows where our greate...,15,2
3,2021-06-18 23:29:21,Looking for examples of stellar charts made so...,15,2
4,2021-06-18 23:05:03,"With #WISQARS Data Visualization, you can disp...",16,2
...,...,...,...,...
33585,2010-01-06 06:13:47,Need data.gov.sg now! RT @jranck: Chart Wars: ...,15,2
33586,2010-01-06 03:24:09,Great presentation by @alexlundry: Chart Wars:...,15,2
33587,2010-01-06 00:29:44,RT @briandevine: u should also check out the d...,15,2
33588,2010-01-05 19:45:15,New Data Visualization - Application performan...,15,2


In [35]:
df = pd.concat([df,df1,df2],axis=0)

In [37]:
df['field'].value_counts()

field
0    241386
1     39116
2     33590
Name: count, dtype: int64

In [38]:
df

,date,tweet,topic,field
0,2021-06-20 05:26:01,What can be done? - Never blindly trust an ab...,15,0
1,2021-06-20 04:46:01,"""We need a paradigm shift from model-centric t...",15,0
2,2021-06-20 04:30:00,Using high-resolution satellite data and compu...,15,0
3,2021-06-20 04:19:01,.@Stephenson_Data shares four steps that will ...,15,0
4,2021-06-20 02:40:01,"""Curricula is inherently brittle in a world wh...",15,0
...,...,...,...,...
33585,2010-01-06 06:13:47,Need data.gov.sg now! RT @jranck: Chart Wars: ...,15,2
33586,2010-01-06 03:24:09,Great presentation by @alexlundry: Chart Wars:...,15,2
33587,2010-01-06 00:29:44,RT @briandevine: u should also check out the d...,15,2
33588,2010-01-05 19:45:15,New Data Visualization - Application performan...,15,2


In [13]:
min([len(i) for i in df.iloc[:900]['tweet']])

42

In [21]:
df.iloc[400:450]['tweet'].to_list()

['#AlmaBetter aims to train 25,000 data science aspirants by March next year to make them employment-ready. It has allocated a Rs 2 crore fund to encourage more students to take up the subject.  @AlmaBetter   https://t.co/Tm74CxywL0',
 "Confirmed: High proportion of women, black and disabled students enrolled on AI &amp; data science courses!  Now: We're seeking companies who want to recruit students for paid placements and help them gain experience + contribute to the sector.  Find out more:  https://t.co/XcZbYhQCDI  https://t.co/xGEjkaQ0a5",
 'Quantum physics both underlies our everyday lives and is a vast and mysterious realm. By comparing theories with experimental data, researchers @Harvard are working to map out the laws that govern the quantum realm:  https://t.co/SmV42BxCcO  https://t.co/59tQj2I0xG',
 ".@CIOdive found that 1 in 5 #data professionals say the infrastructure and systems in place are unable to scale accordingly in response to increased data volume processing needs.

In [18]:
# Setup pipeline on GPU
fe_pipeline = pipeline(
    task="feature-extraction",
    model="vinai/bertweet-base",
    torch_dtype=torch.float16,
    device=0
)

/home/wladek/.local/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [19]:
import torch
from tqdm import tqdm
import gc
import numpy as np
import re
texts = df['tweet'].tolist()
batch_size = 32
output_file = "/media/wladek/Wladyslaw/bertweet_embeddings_2.pt"


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

with open(output_file, "ab") as f:
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = [i for i in texts[i:i + batch_size]]

        # Filter out empty or bad tweets
        batch = [emoji_pattern.sub(r'', text) for text in batch if isinstance(text, str) and text.strip()]
        if not batch:
            continue

        try:
            with torch.no_grad():
                outputs_list = fe_pipeline(batch, padding=True, truncation=True, return_tensors="pt")
                
                embeddings = []
                for j, emb in enumerate(outputs_list):
                    try:
                        emb_tensor = torch.tensor(emb)  # [seq_len, hidden_dim]
                        if emb_tensor.ndim != 3 or emb_tensor.shape[0] < 1:
                            print(f"Skipping bad embedding at index {i + j}, shape={emb_tensor.shape}")
                            continue
                        cls_emb = emb_tensor[0, 0]  # CLS token
                        embeddings.append(cls_emb.detach().cpu())
                    except Exception as inner_e:
                        print(f"Inner error at index {i + j}: {inner_e}")
                        continue

                if not embeddings:
                    continue

                outputs = torch.stack(embeddings).float().cpu()
                f.write(outputs.numpy().tobytes())

        except Exception as e:
            print(f"Error at batch {i}: {e}")
            continue

        # Cleanup
        del outputs_list, outputs, embeddings
        gc.collect()
        torch.cuda.empty_cache()

  0%|                                                  | 0/9816 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_21452/2746591588.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb_tensor = torch.tensor(emb)  # [seq_len, hidden_dim]
  0%|                                       | 13/9816 [00:19<3:50:30,  1.41s/it]../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [64,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [64,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [64,0,0], thread: [98,0,0

Error at batch 416: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 448: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 480: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 512: CUDA error: device-side assert triggered
CUDA kernel errors might

  1%|▏                                        | 55/9816 [00:19<08:05, 20.12it/s]

Error at batch 1088: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 1120: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 1152: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 1184: CUDA error: device-side assert triggered
CUDA kernel errors m

  1%|▎                                        | 77/9816 [00:19<04:18, 37.61it/s]

...

 16%|██████                                | 1572/9816 [00:34<01:18, 105.07it/s]

Error at batch 49888: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 49920: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 49952: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 49984: CUDA error: device-side assert triggered
CUDA kernel erro

 16%|██████▎                                | 1591/9816 [00:34<02:57, 46.30it/s]


Error at batch 50592: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 50624: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 50656: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Error at batch 50688: CUDA error: device-side assert triggered
CUDA kernel erro

KeyboardInterrupt: 

In [6]:
import numpy as np
with open("/media/wladek/Wladyslaw/bertweet_embeddings.pt", "rb") as f:
    # header = np.frombuffer(f.read(8), dtype=np.int32)
    # total_embeddings, embedding_dim = header
    data = f.read()
    embeddings = np.frombuffer(data, dtype=np.float32).reshape(1_600_000, 768)

In [7]:
embeddings.shape

(1600000, 768)

In [20]:
indices = np.random.choice([i for i in range(1_600_000)], size=16_000, replace=False)
embeddings_train = embeddings[indices]

In [22]:
embeddings_train.shape

(16000, 768)

In [25]:
import umap
import numpy as np
import joblib
umap_model = umap.UMAP(
    n_neighbors=5,        # controls local vs global structure
    n_components=3,        # 2D for visualization; use 3 or more if needed
    metric='cosine',       # or 'euclidean'
    n_jobs=-1
).fit(embeddings_train)
result = umap_model.transform(embeddings)
result.dump("umap_result_3d.npy")

In [26]:
result1 = np.load("umap_result_3d.npy", allow_pickle=True)
result1.shape

(1600000, 3)

In [29]:
from pacmap import PaCMAP
pacmap_model = PaCMAP(n_components=2,
                      n_neighbors=10,
                      MN_ratio=0.5,
                      FP_ratio=2.0,
                     save_tree=True)
pacmap_model.fit(embeddings_train)
result = pacmap_model.transform(embeddings)
result.dump("pacmap_result.npy")

In [30]:
result1 = np.load("pacmap_result.npy", allow_pickle=True)
result1.shape

(1600000, 2)

In [31]:
pacmap_model = PaCMAP(n_components=3,
                      n_neighbors=10,
                      MN_ratio=0.5,
                      FP_ratio=2.0,
                     save_tree=True)
pacmap_model.fit(embeddings_train)
result = pacmap_model.transform(embeddings)
result.dump("pacmap_result_3d.npy")

Note: `n_components != 2` have not been thoroughly tested.


In [32]:
result1 = np.load("pacmap_result_3d.npy", allow_pickle=True)
result1.shape

(1600000, 3)

In [36]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=2)
pca_model.fit(embeddings_train)
result = pca_model.transform(embeddings)
result.dump("pca_result.npy")

In [37]:
pca_model = PCA(n_components=3)
pca_model.fit(embeddings_train)
result = pca_model.transform(embeddings)
result.dump("pca_result_3d.npy")

In [41]:
from sklearn.decomposition import IncrementalPCA
pca_model = IncrementalPCA(n_components=2, batch_size=1000)
pca_model.fit(embeddings)
result = pca_model.transform(embeddings)
result.dump("incremental_pca_result.npy")

In [42]:
from sklearn.decomposition import IncrementalPCA
pca_model = IncrementalPCA(n_components=3, batch_size=1000)
pca_model.fit(embeddings)
result = pca_model.transform(embeddings)
result.dump("incremental_pca_result_3d.npy")